In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,5277
2,Huelva,Confirmados PDIA 14 días,2319
3,Huelva,Tasa PDIA 14 días,"454,0444019790775"
4,Huelva,Confirmados PDIA 7 días,1052
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,34
629,Municipio de Huelva sin especificar,Total Confirmados,102
630,Municipio de Huelva sin especificar,Curados,20


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  5277.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1392.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 521 personas en los últimos 7 días 

Un positivo PCR cada 236 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,5277.0,1052.0,2319.0,510743.0,205.974433,454.044402,150.0
Sierra de Huelva-Andévalo Central,836.0,194.0,359.0,67571.0,287.105415,531.293010,59.0
Huelva-Costa,2627.0,493.0,1102.0,288115.0,171.112229,382.486160,59.0
Huelva (capital),1392.0,276.0,610.0,143663.0,192.116272,424.604804,35.0
Condado-Campiña,1712.0,331.0,789.0,155057.0,213.469885,508.845134,30.0
Aracena,181.0,77.0,109.0,8107.0,949.796472,1344.517084,26.0
Valverde del Camino,173.0,58.0,106.0,12820.0,452.418097,826.833073,17.0
Gibraleón,220.0,72.0,146.0,12607.0,571.111287,1158.086777,10.0
Ayamonte,118.0,20.0,45.0,20946.0,95.483625,214.838155,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Escacena del Campo,78.0,23.0,54.0,2284.0,1007.005254,2364.273205,0.0
Villarrasa,66.0,2.0,32.0,2176.0,91.911765,1470.588235,0.0
Aracena,181.0,77.0,109.0,8107.0,949.796472,1344.517084,26.0
Gibraleón,220.0,72.0,146.0,12607.0,571.111287,1158.086777,10.0
Palma del Condado (La),252.0,56.0,113.0,10761.0,520.397733,1050.088282,4.0
Valverde del Camino,173.0,58.0,106.0,12820.0,452.418097,826.833073,17.0
Calañas,32.0,8.0,23.0,2791.0,286.635614,824.077392,0.0
Niebla,54.0,15.0,33.0,4117.0,364.342968,801.554530,1.0
Manzanilla,37.0,7.0,17.0,2135.0,327.868852,796.252927,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villarrasa,66.0,2.0,32.0,2176.0,91.911765,1470.588235,0.0,0.062500
San Juan del Puerto,97.0,2.0,16.0,9300.0,21.505376,172.043011,1.0,0.125000
Rosal de la Frontera,15.0,1.0,7.0,1697.0,58.927519,412.492634,0.0,0.142857
Cortegana,121.0,5.0,24.0,4666.0,107.158165,514.359194,0.0,0.208333
Almonte,278.0,44.0,158.0,24191.0,181.885825,653.135464,2.0,0.278481
Aroche,30.0,5.0,16.0,3073.0,162.707452,520.663846,4.0,0.312500
Villalba del Alcor,70.0,7.0,22.0,3338.0,209.706411,659.077292,1.0,0.318182
Zarza-Perrunal (La),6.0,1.0,3.0,1267.0,78.926598,236.779795,0.0,0.333333
Campofrío,5.0,1.0,3.0,675.0,148.148148,444.444444,0.0,0.333333
